## LINEAR PROGRAMING & PREFERENCES

Linear programming is the technique used to maximize or minimize a function. The idea is to optimize a complex function by best
representing them with linear relationships. In simpler terms, we try to optimize (to maximize or minimize) a function denoted in linear terms and bounded by linear constraints. 

This notebook will use PuLP python package that can be used to solve optimization problems using Linear programming.

In [253]:
#import libraries
from pulp import *
import pandas as pd
import numpy as np
from scipy import stats

### Exercise 1: Profit Maximazation

A toy manufacturing organization manufactures two types of toys A and B. Both the toys are sold at 25EUR0 and 20 EURO respectively.
There are 2000 resource units available every day from which the toy A requires 20 units while toy B requires 12 units. Both of these toys require a production time of 5 minutes. Total working hours are 9 hours a day. 

What should be the manufacturing quantity for each of the pipes to maximize the profits ?

In [254]:
#Initialize the problem, since we nee to maximize the profit we will use LpMaximize
prob_profit = LpProblem('Profit_Maximize',LpMaximize) # Model

In [255]:
#Defining problem variables
x=LpVariable("Toy1_quantity",0,None,LpInteger) 
y=LpVariable("Toy2_quantity",0,None,LpInteger)

In [256]:
#Objective function
prob_profit += 25*x + 20*y

#Constraints
prob_profit += 20*x + 12*y <= 2000 #constraints for resources
prob_profit += 5*x + 5*y <= 540    #constraints for time: 9 hours = 540min

prob_profit.writeLP("ToyProduction.lp")
prob_profit.solve()


1

#### Status code

There are 5 status codes:

    1. Not Solved: Status prior to solving the problem.
    2. Optimal: An optimal solution has been found.
    3. Infeasible: There are no feasible solutions (e.g. if you set the constraints x <= 1 and x >=2).
    4. Unbounded: The constraints are not bounded, maximising the solution will tend towards infinity.
    5. Undefined: The optimal solution may exist but may not have been found.



In [257]:
#Checking the status of the problem
LpStatus[prob_profit.status]

'Optimal'

In [258]:
print("To maximize the profit of manufacturing organization, we need the following quantity:" )

for v in prob_profit.variables():
    print(v.name,"=",v.varValue)

print ("The maximized profit will be = ",value(prob_profit.objective) )


To maximize the profit of manufacturing organization, we need the following quantity:
Toy1_quantity = 88
Toy2_quantity = 20
The maximized profit will be =  2600


### Exercise 2: How to visit Paris ?

John Doe, an American researcher, goes to Paris to present an article at a conference. It is his first time in the French capital. He arrives on Monday and he intends to spend the whole week at the conference. However, the conference ends on Friday, and Mr. Doe leaves only on Sunday in the end evening. So he has the opportunity to visit the city during the weekend. He gives himself 12 hours on the two days to fully visit Paris with a maximum budget of 65 EURO.

By doing his own research (via some social medias for instance), he finds that it is interesting to visit : La Tour Eiffel (TE), Le Musée du louvre (ML), l’Arc de triomphe (AT), le Musée d’Orsay (MO), le Jardin des tuileries (JT), les Catacombes (CA), le Centre Pompidou(CP), la Cathédrale Notre Dame de Paris (CN), la Basilique du Sacré-Coeur (BS), la Sainte Chapelle (SC), La Place de la Concorde (PC), la Tour Montparnasse (TM) and l’Avenue des Champs-Elysées (AC).

In [259]:
site_names=['TE','ML','AT','MO','JT','CA','CP','CN','BS','SC','PC','TM','AC'] #site name
duration= [9/2, 3, 1, 2, 3/2, 2, 5/2, 2, 2, 3/2, 3/4, 2, 3/2] #duration in hours
price= [15.50, 12, 9.50, 11, 0, 10, 10, 5, 8, 8.50, 0, 15, 0] #price in euro   
    
max_budget = 65 #price in euro
max_duration = 12 #hours
    
#defining problem variables
p_var = [LpVariable(site_name, 0, 1,LpInteger) for site_name in site_names]
    
#places informations
sites_info ={i: j for i, j in zip(site_names,p_var)}

In [260]:
def initial_constraints(p):


    #object function
    p += lpSum([p_var[i] for i in range(len(p_var))])

    #constraints


    p += lpSum([p_var[i] * price[i] for i in range(len(p_var))]) <= max_budget

    p += lpSum([p_var[i] * duration[i] for i in range(len(p_var))]) <= max_duration 
    

#### 2.1 What is the maximum number of site Mr Doe can visit? 
It is assumed that Mr. Doe gives equal importance to each tourist site, and he wants to visit the maximum number of sites. Which
list(s) of places could you recommend to him ? This solution will be called ListVisit 1.

In [261]:
#initiazlizing the problem
prob_site = LpProblem('RecommendSites',LpMaximize)
#call a function
initial_constraints(prob_site)

prob_site.writeLP("RecommendSites.lp")

prob_site.solve()
LpStatus[prob_site.status]

'Optimal'

In [262]:
#creating an empty list to store the sites
listVisit1=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in prob_site.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listVisit1.append(v.name) # store all the recommended sites

#print("The list of maximun visited sites:",listVisit1)

print ("The maximum number of sites Mr Doe can visit is = ",value(prob_site.objective) )
print("The list of recommended sites is: ", listVisit1)

AC = 1
AT = 1
BS = 1
CA = 1
CN = 0
CP = 0
JT = 1
ML = 0
MO = 0
PC = 0
SC = 1
TE = 0
TM = 1
The maximum number of sites Mr Doe can visit is =  7
The list of recommended sites is:  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']


#### 2.2 Recommended sites based on Mr Doe Preferences

#### Preference 1: 
If two sites are geographically very close (within a radius of 1 km of walking), he will prefer to visit these two sites instead of visiting only one. 

In [263]:
#creating a matrix to store distance
site = [name for name in site_names]
data = [[0, 3.8, 2.1, 2.4, 3.5, 4.2, 5.0,  4.4, 5.5, 4.2, 2.5, 3.1, 1.9],
        [0,   0, 3.8, 1.1, 1.3, 3.3, 1.3,  1.1, 3.4, 0.800, 1.7, 2.5, 2.8],
        [0,   0,   0, 3.1, 3.0, 5.8, 4.8,  4.9, 4.3, 4.6, 2.2, 4.4, 1.0],
        [0,   0,   0,   0, 0.900, 3.1, 2.5,  2.0, 3.9, 1.8, 1.0, 2.3, 2.1],
        [0,   0,   0,   0,   0, 4.2, 2.0,  2.4, 2.7, 2.0, 1.0, 3.4, 2.1],
        [0,   0,   0,   0,   0,   0, 3.5,  2.7, 6.5, 2.6, 3.8, 1.3, 4.9],
        [0,   0,   0,   0,   0,   0,   0, 0.850, 3.7, 0.900, 2.7, 3.4, 3.8],
        [0,   0,   0,   0,   0,   0,   0,    0, 4.5, 0.400, 2.8, 2.7, 3.9],
        [0,   0,   0,   0,   0,   0,   0,    0,   0, 4.2, 3.3, 5.7, 3.8],
        [0,   0,   0,   0,   0,   0,   0,    0,   0,   0, 2.5, 2.6, 3.6],
        [0,   0,   0,   0,   0,   0,   0,    0,   0,   0,   0, 3.0, 1.2],
        [0,   0,   0,   0,   0,   0,   0,    0,   0,   0,   0,   0, 2.1],
        [0,   0,   0,   0,   0,   0,   0,    0,   0,   0,   0,   0,   0]]
    
distance = pd.DataFrame(np.matrix(data), columns=site, index=site) #distance in km
distance

,TE,ML,AT,MO,JT,CA,CP,CN,BS,SC,PC,TM,AC
TE,0.0,3.8,2.1,2.4,3.5,4.2,5.0,4.40,5.5,4.2,2.5,3.1,1.9
ML,0.0,0.0,3.8,1.1,1.3,3.3,1.3,1.10,3.4,0.8,1.7,2.5,2.8
AT,0.0,0.0,0.0,3.1,3.0,5.8,4.8,4.90,4.3,4.6,2.2,4.4,1.0
MO,0.0,0.0,0.0,0.0,0.9,3.1,2.5,2.00,3.9,1.8,1.0,2.3,2.1
JT,0.0,0.0,0.0,0.0,0.0,4.2,2.0,2.40,2.7,2.0,1.0,3.4,2.1
CA,0.0,0.0,0.0,0.0,0.0,0.0,3.5,2.70,6.5,2.6,3.8,1.3,4.9
CP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.85,3.7,0.9,2.7,3.4,3.8
CN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,4.5,0.4,2.8,2.7,3.9
BS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,4.2,3.3,5.7,3.8
SC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,2.5,2.6,3.6


In [264]:
#constraints of preference 1
def preference1(p):
    #checking the sites that are in 1km
    for i, site_name1 in enumerate(site):
        for j, site_name2 in enumerate(site):
            if (distance.loc[site_name1, site_name2] > 0 and distance.loc[site_name1, site_name2] <= 1 and site_name1 != site_name2):
                p += sites_info[site[i]] == sites_info[site[j]]

In [265]:
#initiazlizing the problem
prefer1 = LpProblem('close_site',LpMaximize)

#initial constraints
initial_constraints(prefer1)

#additional constraints
preference1(prefer1)

prefer1.writeLP("close_site.lp")
prefer1.solve()
LpStatus[prefer1.status]

'Optimal'

In [266]:
listPref1=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in prefer1.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref1.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 1 is  = ",value(prefer1.objective) )
print("The list of recommended sites is: ", listPref1)

AC = 1
AT = 1
BS = 1
CA = 0
CN = 0
CP = 0
JT = 1
ML = 0
MO = 1
PC = 1
SC = 0
TE = 0
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 1 is  =  7
The list of recommended sites is:  ['AC', 'AT', 'BS', 'JT', 'MO', 'PC', 'TM']


#### Preference 2: He absolutely wants to visit the Eiffel Tower (TE) and Catacombes (CA). 

Here the constraints is that, the value of TE and CA should be 1

In [267]:
def preference2(p):
    p += sites_info['TE'] == 1 
    p += sites_info['CA'] == 1 
    

In [268]:
#initiazlizing the problem
prefer2 = LpProblem('MustVisitTEandCA',LpMaximize)

#initial constraints
initial_constraints(prefer2)

#additional constraints constrainst
preference2(prefer2)

prob_site.writeLP("MustVisitTEandCA.lp")

prefer2.solve()
LpStatus[prefer2.status]

'Optimal'

In [269]:
listPref2=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in prefer2.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref2.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 2 is  = ",value(prefer2.objective) )
print("The list of recommended sites is: ", listPref2)

AC = 1
AT = 1
BS = 0
CA = 1
CN = 0
CP = 0
JT = 0
ML = 0
MO = 0
PC = 1
SC = 1
TE = 1
TM = 0
The maximum number of sites Mr Doe can visit based on his preference 2 is  =  6
The list of recommended sites is:  ['AC', 'AT', 'CA', 'PC', 'SC', 'TE']


#### Preference 3: If he visits Notre Dame Cathedral (CN) then he will not visit the Sainte Chapelle (SC).
Here the additional constraints is that, if the value of CN is 1 then SC should be 0, meaning that they should not have the same value.

In [270]:
def preference3(p):
    p += sites_info['CN'] != sites_info['SC']
    

In [271]:
#initiazlizing the problem
prefer3 = LpProblem('EitherCNorSC',LpMaximize)

#initial constraints
initial_constraints(prefer3)

#additional constraints constrainst
preference3(prefer3)

prefer3.writeLP("EitherCNorSC.lp")

prefer3.solve()
LpStatus[prefer3.status]

'Optimal'

In [272]:
listPref3=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in prefer3.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref3.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 3 is  = ",value(prefer3.objective) )
print("The list of recommended sites is: ", listPref3)

AC = 1
AT = 1
BS = 1
CA = 1
CN = 0
CP = 0
JT = 1
ML = 0
MO = 0
PC = 0
SC = 1
TE = 0
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 3 is  =  7
The list of recommended sites is:  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']


#### Preference 4 : He absolutely wants to visit Tour Montparnasse (TM).
The value of TM must be 1

In [273]:
def preference4(p):
    p += sites_info['TM'] == 1
    

In [274]:
#initiazlizing the problem
prefer4 = LpProblem('MustVisitTM',LpMaximize)

#initial constraints
initial_constraints(prefer4)

#additional constraints constrainst
preference4(prefer4)

prefer4.writeLP("MustVisitTM.lp")

prefer4.solve()
LpStatus[prefer4.status]

'Optimal'

In [275]:
listPref4=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in prefer4.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref4.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 4 is  = ",value(prefer4.objective) )
print("The list of recommended sites is: ", listPref4)

AC = 1
AT = 1
BS = 1
CA = 0
CN = 0
CP = 0
JT = 1
ML = 0
MO = 1
PC = 0
SC = 1
TE = 0
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 4 is  =  7
The list of recommended sites is:  ['AC', 'AT', 'BS', 'JT', 'MO', 'SC', 'TM']


#### Preference 5 : If he visits the Louvre (ML) Museum then he must visit the Pompidou Center (CP).
If the value of ML is 1 then CP should be 1 too, otherwise he will not visit any of them!

In [276]:
def preference5(p):
    p += sites_info['ML'] == sites_info['CP']
    

In [277]:
#initiazlizing the problem
prefer5 = LpProblem('MlandCP',LpMaximize)

#initial constraints
initial_constraints(prefer5)

#additional constraints constrainst
preference5(prefer5)

prefer5.writeLP("MlandCP.lp")

prefer5.solve()
LpStatus[prefer5.status]

'Optimal'

In [278]:
listPref5=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in prefer5.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref5.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 5 is  = ",value(prefer5.objective) )
print("The list of recommended sites is: ", listPref5)

AC = 1
AT = 1
BS = 1
CA = 0
CN = 1
CP = 0
JT = 1
ML = 0
MO = 0
PC = 0
SC = 1
TE = 0
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 5 is  =  7
The list of recommended sites is:  ['AC', 'AT', 'BS', 'CN', 'JT', 'SC', 'TM']


#### 2.2 (a) For each of the five preferences above, suggest to Mr. Doe, one or more lists of tourist sites to visit. 
Are the obtained lists different from the solution ListVisit 1 ? To answer this last question, you can implement a python function returning True (respectively False) if two lists are identical (respectively different).

In [279]:
#to check if two lists are identical
def is_identical(list1, list2):
    return (sorted(list1) == sorted(list2)) # will return true or false

In [280]:
#def list_comparisons:
FivePreferences=[listPref1, listPref2,listPref3,listPref4,listPref5]
for i in range(len(FivePreferences)):
    if(is_identical(listVisit1, FivePreferences[i])):
        print("listVisit1 and the solutions of Preference " + str(i + 1) + " are identical")
        print("Listvisit1: ",listVisit1)
        print("Mr Doe can visit sites of the list obtained in Preference " + str(i+1) ,FivePreferences[i])       


listVisit1 and the solutions of Preference 3 are identical
Listvisit1:  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']
Mr Doe can visit sites of the list obtained in Preference 3 ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']


#### 2.2(b) If Mr. Doe wishes, at the same time, to take into account Preference 1 and Preference 2, which list(s) would you recommend to him ?

In [281]:
#initiazlizing the problem
p12 = LpProblem('pref1and2',LpMaximize)

#initial constraints
initial_constraints(p12)

#additional constraints constrainst
preference1(p12) #constraints of preference 1
preference2(p12) #constraints of preference 2

p12.writeLP("pref1and2.lp")

p12.solve()
LpStatus[p12.status]

'Optimal'

In [282]:
listPref12=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p12.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref12.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 1 and 2 = ",value(p12.objective) )
print("The list of recommended sites is: ", listPref12)

AC = 0
AT = 0
BS = 0
CA = 1
CN = 0
CP = 0
JT = 1
ML = 0
MO = 1
PC = 1
SC = 0
TE = 1
TM = 0
The maximum number of sites Mr Doe can visit based on his preference 1 and 2 =  5
The list of recommended sites is:  ['CA', 'JT', 'MO', 'PC', 'TE']


#### 2.2(c) If Mr. Doe wishes, at the same time, to take into account Preference 1 and Preference 3, which list(s) would you recommend to him ?

In [283]:
#initiazlizing the problem
p13 = LpProblem('pref1and3',LpMaximize)

#initial constraints
initial_constraints(p13)

#additional constraints constrainst
preference1(p13) #constraints of preference 1
preference3(p13) #constraints of preference 3

p13.writeLP("pref1and3.lp")

p13.solve()
LpStatus[p13.status]

'Optimal'

In [284]:
listPref13=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p13.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref13.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 1 and 3 = ",value(p13.objective) )
print("The list of recommended sites is: ", listPref13)

AC = 1
AT = 1
BS = 1
CA = 0
CN = 0
CP = 0
JT = 1
ML = 0
MO = 1
PC = 1
SC = 0
TE = 0
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 1 and 3 =  7
The list of recommended sites is:  ['AC', 'AT', 'BS', 'JT', 'MO', 'PC', 'TM']


#### 2.2(d) If Mr. Doe wishes, at the same time, to take into account Preference 1 and Preference 4, which list(s) would you recommend to him ? 

In [285]:
#initiazlizing the problem
p14 = LpProblem('pref1and4',LpMaximize)

#initial constraints
initial_constraints(p14)

#additional constraints constrainst
preference1(p14) #constraints of preference 1
preference4(p14) #constraints of preference 4

p14.writeLP("pref1and4.lp")

p14.solve()
LpStatus[p14.status]

'Optimal'

In [286]:
listPref14=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p14.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref14.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 1 and 4 = ",value(p14.objective) )
print("The list of recommended sites is: ", listPref14)

AC = 1
AT = 1
BS = 0
CA = 1
CN = 0
CP = 0
JT = 1
ML = 0
MO = 1
PC = 1
SC = 0
TE = 0
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 1 and 4 =  7
The list of recommended sites is:  ['AC', 'AT', 'CA', 'JT', 'MO', 'PC', 'TM']


#### 2.2(e) If Mr. Doe wishes, at the same time, to take into account Preference 2 and Preference 5, which list(s) would you recommend to him ?

In [287]:
#initiazlizing the problem
p25 = LpProblem('pref2and5',LpMaximize)

#initial constraints
initial_constraints(p25)

#additional constraints constrainst
preference2(p25) #constraints of preference 2
preference5(p25) #constraints of preference 5

p25.writeLP("pref2and5.lp")

p25.solve()
LpStatus[p25.status]

'Optimal'

In [288]:
listPref25=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p25.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref25.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 2 and 5= ",value(p25.objective) )
print("The list of recommended sites is: ", listPref25)

AC = 1
AT = 1
BS = 0
CA = 1
CN = 0
CP = 0
JT = 0
ML = 0
MO = 0
PC = 1
SC = 1
TE = 1
TM = 0
The maximum number of sites Mr Doe can visit based on his preference 2 and 5=  6
The list of recommended sites is:  ['AC', 'AT', 'CA', 'PC', 'SC', 'TE']


#### 2.2(f) If Mr. Doe wishes, at the same time, to take into account Preference 3 and Preference 4, which list(s) would you recommend to him ?

In [289]:
#initiazlizing the problem
p34 = LpProblem('pref3and4',LpMaximize)

#initial constraints
initial_constraints(p34)

#additional constraints constrainst
preference3(p34) #constraints of preference 3
preference4(p34) #constraints of preference 4

p34.writeLP("pref3and4.lp")

p34.solve()
LpStatus[p34.status]

'Optimal'

In [290]:
listPref34=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p34.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref34.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 3 and 4 = ",value(p34.objective) )
print("The list of recommended sites is: ", listPref34)

AC = 1
AT = 1
BS = 1
CA = 0
CN = 0
CP = 0
JT = 1
ML = 0
MO = 1
PC = 0
SC = 1
TE = 0
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 3 and 4 =  7
The list of recommended sites is:  ['AC', 'AT', 'BS', 'JT', 'MO', 'SC', 'TM']


#### 2.2(g) If Mr. Doe wishes, at the same time, to take into account Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [291]:
#initiazlizing the problem
p45= LpProblem('pref4and5',LpMaximize)

#initial constraints
initial_constraints(p45)

#additional constraints constrainst
preference4(p45) #constraints of preference 4
preference5(p45) #constraints of preference 5

p45.writeLP("pref4and5.lp")

p45.solve()
LpStatus[p45.status]

'Optimal'

In [292]:
listPref45=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p45.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref45.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 4 and 5 = ",value(p45.objective) )
print("The list of recommended sites is: ", listPref45)

AC = 1
AT = 1
BS = 0
CA = 1
CN = 0
CP = 0
JT = 1
ML = 0
MO = 1
PC = 0
SC = 1
TE = 0
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 4 and 5 =  7
The list of recommended sites is:  ['AC', 'AT', 'CA', 'JT', 'MO', 'SC', 'TM']


#### 2.2 (h) If Mr. Doe wishes, at the same time, to take into account Preference 1, Preference 2 and Preference 4, which list(s) would you recommend to him ?

In [293]:
#initiazlizing the problem
p124= LpProblem('pref124',LpMaximize)

#initial constraints
initial_constraints(p124)

#additional constraints constrainst           
preference1(p124) #constraints of preference 1
preference2(p124) #constraints of preference 2
preference4(p124) #constraints of preference 3

p124.writeLP("pref124.lp")

p124.solve()
LpStatus[p124.status]

'Optimal'

In [294]:
listPref124=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p124.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref124.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 1,2 and 4 = ",value(p124.objective) )
print("The list of recommended sites is: ", listPref124)

AC = 1
AT = 1
BS = 0
CA = 1
CN = 0
CP = 0
JT = 0
ML = 0
MO = 0
PC = 0
SC = 0
TE = 1
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 1,2 and 4 =  5
The list of recommended sites is:  ['AC', 'AT', 'CA', 'TE', 'TM']


#### 2.2 (i) If Mr. Doe wishes, at the same time, to take into account Preference 2, Preference 3 and Preference 5, which list(s) would you recommend to him ?

In [295]:
#initiazlizing the problem
p235= LpProblem('pref235',LpMaximize)

#initial constraints
initial_constraints(p235)

#additional constraints constrainst           
preference2(p235) #constraints of preference 2
preference3(p235) #constraints of preference 3
preference5(p235) #constraints of preference 5

p235.writeLP("pref235.lp")

p235.solve()
LpStatus[p235.status]

'Optimal'

In [296]:
listPref235=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p235.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref235.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 2,3 and 5 = ",value(p235.objective) )
print("The list of recommended sites is: ", listPref235)

AC = 1
AT = 1
BS = 0
CA = 1
CN = 0
CP = 0
JT = 0
ML = 0
MO = 0
PC = 1
SC = 1
TE = 1
TM = 0
The maximum number of sites Mr Doe can visit based on his preference 2,3 and 5 =  6
The list of recommended sites is:  ['AC', 'AT', 'CA', 'PC', 'SC', 'TE']


#### 2.2 (j) If Mr. Doe wishes, at the same time, to take into account Preference 2, Preference 3, Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [297]:
#initiazlizing the problem
p2345= LpProblem('pref2345',LpMaximize)

#initial constraints
initial_constraints(p2345)

#additional constraints constrainst           
preference2(p2345) #constraints of preference 2
preference3(p2345) #constraints of preference 3
preference4(p2345) #constraints of preference 4
preference5(p2345) #constraints of preference 5

p2345.writeLP("pref2345.lp")

p2345.solve()
LpStatus[p2345.status]

'Optimal'

In [298]:
listPref2345=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p2345.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref2345.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 2, 3, 4 and 5 = ",value(p2345.objective) )
print("The list of recommended sites is: ", listPref2345)

AC = 1
AT = 1
BS = 0
CA = 1
CN = 0
CP = 0
JT = 0
ML = 0
MO = 0
PC = 1
SC = 0
TE = 1
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 2, 3, 4 and 5 =  6
The list of recommended sites is:  ['AC', 'AT', 'CA', 'PC', 'TE', 'TM']


#### 2.2 (k) If Mr. Doe wishes, at the same time, to take into account Preference 1, Preference 2, Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [299]:
#initiazlizing the problem
p1245= LpProblem('pref1245',LpMaximize)

#initial constraints
initial_constraints(p1245)

#additional constraints constrainst           
preference1(p1245) #constraints of preference 1
preference2(p1245) #constraints of preference 2
preference4(p1245) #constraints of preference 4
preference5(p1245) #constraints of preference 5

p1245.writeLP("pref1245.lp")

p1245.solve()
LpStatus[p1245.status]

'Optimal'

In [300]:
listPref1245=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p1245.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref1245.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 1,2,4 and 5 = ",value(p1245.objective) )
print("The list of recommended sites is: ", listPref1245)

AC = 1
AT = 1
BS = 0
CA = 1
CN = 0
CP = 0
JT = 0
ML = 0
MO = 0
PC = 0
SC = 0
TE = 1
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 1,2,4 and 5 =  5
The list of recommended sites is:  ['AC', 'AT', 'CA', 'TE', 'TM']


#### 2.2(l) If Mr. Doe wishes, at the same time, to take into account Preference 1, Preference 2, Preference 3, Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [301]:
#initiazlizing the problem
p12345= LpProblem('pref12345',LpMaximize)

#initial constraints
initial_constraints(p12345)

#additional constraints constrainst           
preference1(p12345) #constraints of preference 1
preference2(p12345) #constraints of preference 2
preference3(p12345) #constraints of preference 3
preference4(p12345) #constraints of preference 4
preference5(p12345) #constraints of preference 5

p12345.writeLP("p12345.lp")

p12345.solve()
LpStatus[p12345.status]

'Optimal'

In [302]:
listPref12345=[]

#printing the values of each site, 0 means not recommended and 1 means recommended
for v in p12345.variables():
    print(v.name,"=",v.varValue)
    if(v.varValue==1.0):
        listPref12345.append(v.name) # store all the recommended sites

print ("The maximum number of sites Mr Doe can visit based on his preference 1, 2, 3, 4 and 5 = ",value(p12345.objective) )
print("The list of recommended sites is: ", listPref12345)

AC = 1
AT = 1
BS = 0
CA = 1
CN = 0
CP = 0
JT = 0
ML = 0
MO = 0
PC = 0
SC = 0
TE = 1
TM = 1
The maximum number of sites Mr Doe can visit based on his preference 1, 2, 3, 4 and 5 =  5
The list of recommended sites is:  ['AC', 'AT', 'CA', 'TE', 'TM']


#### 2.2(m) Is the solution ListVisit1 different to these solutions founded above (with the combination of preferences) ?

In [303]:
listCombinations=[listPref12,listPref13,listPref14,listPref25,listPref34,listPref45,
                  listPref124,listPref235,listPref2345,listPref1245,listPref12345]

for i in range(len(listCombinations)):
    if(is_identical(listVisit1, listCombinations[i])):
        print("No, The solution of the list is the same solution of listvisit1: ")
        print(listVisit1)
        print(listCombinations[i])
    else:
        print("Listvisit1 : ",listVisit1)
        print("Yes, The solution of Listvisit1 is different from the list: ",listCombinations[i])
        print("================================================================================================")

Listvisit1 :  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']
Yes, The solution of Listvisit1 is different from the list:  ['CA', 'JT', 'MO', 'PC', 'TE']
Listvisit1 :  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']
Yes, The solution of Listvisit1 is different from the list:  ['AC', 'AT', 'BS', 'JT', 'MO', 'PC', 'TM']
Listvisit1 :  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']
Yes, The solution of Listvisit1 is different from the list:  ['AC', 'AT', 'CA', 'JT', 'MO', 'PC', 'TM']
Listvisit1 :  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']
Yes, The solution of Listvisit1 is different from the list:  ['AC', 'AT', 'CA', 'PC', 'SC', 'TE']
Listvisit1 :  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']
Yes, The solution of Listvisit1 is different from the list:  ['AC', 'AT', 'BS', 'JT', 'MO', 'SC', 'TM']
Listvisit1 :  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', 'TM']
Yes, The solution of Listvisit1 is different from the list:  ['AC', 'AT', 'CA', 'JT', 'MO', 'SC', 'TM']
Listvisit1 :  ['AC', 'AT', 'BS', 'CA', 'JT', 'SC', '

### 3: Ranking of Touristic Sites
• Dur the ranking of the touristic sites obtained by observing only the Duration criterion (see the column “Duration” of the Table above)

• App the ranking of the touristic sites obtained by observing only the Appreciations criterion (see the column “Appreciations”
of the Table above)

• Pri the ranking of the touristic sites obtained by observing only the Price criterion (see the column “Price” of the Table
above)

Are these rankings two rankings different ? To answer this question, you can use the Kendall 1 or Spearman 2 rank correlation
coefficient.

In [304]:
dur=[9/2,3,1,2,3/2,2,5/2,2,2,3/2,3/4,2,3/2]
app=[5,4,3,2,3,4,1,5,4,1,3,2,5]
pri=[15.5,12,9.5,11,0,10,10,5,8,8.5,0,15,0]

#defining a function to calculate correlations
def spearman_correlation_coefficient(x,y):
    #calculate Spearman Rank correlation and corresponding p-value
    rho, pval = stats.spearmanr(x, y)
    print("Spearman rank correlation") 
    print(rho)
    print("Spearman rank p-value")
    print(pval)
    
    if (rho>0.5): 
        print("Monotonically increasing relationship: Highly correlated")
        
    if(rho<=0.5 and rho >0):
        print("Weak monotonically increasing relationship : Weakly correlated")
        
    if (rho <=0):
        if (rho==0):
            print("Non-monotonic relation")
        else:
            print("monotonically decreasing relationship: Negativly correlated")



In [305]:
print("Correlation between duration and appreciation: ")
spearman_correlation_coefficient(dur,app)

Correlation between duration and appreciation: 
Spearman rank correlation
0.2058187176566192
Spearman rank p-value
0.4999321648617373
Weak monotonically increasing relationship : Weakly correlated


In [306]:
print("Correlation between duration and price: ")
spearman_correlation_coefficient(dur,pri)

Correlation between duration and price: 
Spearman rank correlation
0.7413325704498833
Spearman rank p-value
0.0037316379930495526
Monotonically increasing relationship: Highly correlated


In [307]:
print("Correlation between price and appreciation: ")
spearman_correlation_coefficient(pri,app)

Correlation between price and appreciation: 
Spearman rank correlation
-0.13400233411084597
Spearman rank p-value
0.662512282187264
monotonically decreasing relationship: Negativly correlated
